# hospital data curation project
## phase 4: data integration and merging

merge cleaned datasets to create comprehensive analytical datasets:
- merge patients + visits
- integrate diagnoses and medications
- handle conflicts and resolve entities
- create master dataset for analytics

In [1]:
# import required libraries
import sys
import os
from pathlib import Path

# add src directory to python path
notebook_dir = Path(os.getcwd())
src_dir = notebook_dir / 'src'
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import warnings
warnings.filterwarnings('ignore')

# import project modules
import config
import utils

# use imported modules
CLEANED_DATA_DIR = config.CLEANED_DATA_DIR
PREPROCESSED_DATA_DIR = config.PREPROCESSED_DATA_DIR
LOGS_DIR = config.LOGS_DIR
setup_logging = utils.setup_logging
print_section_header = utils.print_section_header
save_dataframe = utils.save_dataframe
load_dataframe = utils.load_dataframe


## 1. load cleaned datasets

In [2]:
# setup logging
logger = setup_logging()

# load all cleaned datasets
print_section_header("loading cleaned datasets")

patients_df = load_dataframe(CLEANED_DATA_DIR / 'clean_patients.csv')
visits_df = load_dataframe(CLEANED_DATA_DIR / 'clean_visits.csv')
diagnoses_df = load_dataframe(CLEANED_DATA_DIR / 'clean_diagnoses.csv')
medications_df = load_dataframe(CLEANED_DATA_DIR / 'clean_medications.csv')
staff_df = load_dataframe(CLEANED_DATA_DIR / 'clean_staff.csv')

print("\ndatasets loaded successfully:")
print(f"  patients: {len(patients_df)} rows")
print(f"  visits: {len(visits_df)} rows")
print(f"  diagnoses: {len(diagnoses_df)} rows")
print(f"  medications: {len(medications_df)} rows")
print(f"  staff: {len(staff_df)} rows")

2025-11-10 17:48:27,913 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_patients.csv: 3000 rows, 7 columns
2025-11-10 17:48:27,958 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_visits.csv: 2481 rows, 8 columns
2025-11-10 17:48:27,958 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_visits.csv: 2481 rows, 8 columns
2025-11-10 17:48:27,995 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_diagnoses.csv: 7998 rows, 5 columns
2025-11-10 17:48:27,995 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_diagnoses.csv: 7998 rows, 5 columns
2025-11-10 17:48:28,028 - root - INFO - loaded dataframe from d:\Github Desktop\Python\Hospital Data Curation\data\cleaned\clean_medications.csv: 5324 rows, 6 columns
2025-11-10 17:4


                            loading cleaned datasets                            


datasets loaded successfully:
  patients: 3000 rows
  visits: 2481 rows
  diagnoses: 7998 rows
  medications: 5324 rows
  staff: 500 rows

datasets loaded successfully:
  patients: 3000 rows
  visits: 2481 rows
  diagnoses: 7998 rows
  medications: 5324 rows
  staff: 500 rows


## 2. merge patients and visits

In [3]:
# merge patients with visits data
print_section_header("merging patients and visits")

# check for patient_id consistency
print("checking referential integrity...")
orphaned_visits = ~visits_df['patient_id'].isin(patients_df['patient_id'])
print(f"  orphaned visits (no matching patient): {orphaned_visits.sum()}")

if orphaned_visits.sum() > 0:
    print(f"  removing {orphaned_visits.sum()} orphaned visits")
    visits_df = visits_df[~orphaned_visits]

# merge patients and visits
patient_visits = pd.merge(
    patients_df,
    visits_df,
    on='patient_id',
    how='inner',
    suffixes=('_patient', '_visit')
)

print(f"\nmerged patient-visits dataset: {len(patient_visits)} rows")
print(f"columns: {len(patient_visits.columns)}")

# display sample
print("\nsample merged data:")
print(patient_visits.head(3))


                          merging patients and visits                           

checking referential integrity...
  orphaned visits (no matching patient): 0

merged patient-visits dataset: 2481 rows
columns: 14

sample merged data:
  patient_id              name         dob   gender contact_number  \
0     P10003      Susan Wagner  1965-03-01  Unknown  (372)730-3040   
1     P10005  Theodore Mcgrath  2009-12-26     Male   321.659.7442   
2     P10005  Theodore Mcgrath  2009-12-26     Male   321.659.7442   

                               email  \
0  nicolehernandez@banks-higgins.net   
1               kmorales@hotmail.com   
2               kmorales@hotmail.com   

                                            address visit_id admission_date  \
0  4207 Mora Centers Suite 151, Lake Paul, WI 58477   V22056     2024-02-17   
1   122 Taylor Station, South Jeffreyfurt, DE 96964   V20238     2024-06-22   
2   122 Taylor Station, South Jeffreyfurt, DE 96964   V21235     2024-07-15   

  disc

## 3. aggregate diagnoses per visit

In [4]:
# aggregate diagnoses information
print_section_header("aggregating diagnoses")

# count diagnoses per visit
diagnoses_count = diagnoses_df.groupby('visit_id').size().reset_index(name='diagnosis_count')

# get primary diagnosis (first one per visit)
primary_diagnosis = diagnoses_df.groupby('visit_id').first().reset_index()
primary_diagnosis = primary_diagnosis[['visit_id', 'icd_code']]
primary_diagnosis.rename(columns={'icd_code': 'primary_diagnosis'}, inplace=True)

# get all diagnosis codes as comma-separated list
all_diagnoses = diagnoses_df.groupby('visit_id')['icd_code'].apply(
    lambda x: ', '.join(x.astype(str))
).reset_index()
all_diagnoses.rename(columns={'icd_code': 'all_diagnoses'}, inplace=True)

print(f"diagnosis aggregations created:")
print(f"  visits with diagnoses: {len(diagnoses_count)}")
print(f"  average diagnoses per visit: {diagnoses_count['diagnosis_count'].mean():.2f}")


                             aggregating diagnoses                              

diagnosis aggregations created:
  visits with diagnoses: 3977
  average diagnoses per visit: 2.01
diagnosis aggregations created:
  visits with diagnoses: 3977
  average diagnoses per visit: 2.01


## 4. aggregate medications per visit

In [5]:
# aggregate medications information
print_section_header("aggregating medications")

# count medications per visit
medications_count = medications_df.groupby('visit_id').size().reset_index(name='medication_count')

# get all medications as comma-separated list
all_medications = medications_df.groupby('visit_id')['medication_name'].apply(
    lambda x: ', '.join(x.astype(str))
).reset_index()
all_medications.rename(columns={'medication_name': 'all_medications'}, inplace=True)

print(f"medication aggregations created:")
print(f"  visits with medications: {len(medications_count)}")
print(f"  average medications per visit: {medications_count['medication_count'].mean():.2f}")


                            aggregating medications                             

medication aggregations created:
  visits with medications: 3470
  average medications per visit: 1.53
medication aggregations created:
  visits with medications: 3470
  average medications per visit: 1.53


## 5. create comprehensive master dataset

In [6]:
# merge all components into master dataset
print_section_header("creating master dataset")

# start with patient_visits
master_df = patient_visits.copy()
print(f"starting with patient-visits: {len(master_df)} rows")

# merge diagnosis count
master_df = pd.merge(
    master_df,
    diagnoses_count,
    on='visit_id',
    how='left'
)
master_df['diagnosis_count'].fillna(0, inplace=True)
print(f"after merging diagnosis count: {len(master_df)} rows")

# merge primary diagnosis
master_df = pd.merge(
    master_df,
    primary_diagnosis,
    on='visit_id',
    how='left'
)
print(f"after merging primary diagnosis: {len(master_df)} rows")

# merge all diagnoses
master_df = pd.merge(
    master_df,
    all_diagnoses,
    on='visit_id',
    how='left'
)
print(f"after merging all diagnoses: {len(master_df)} rows")

# merge medication count
master_df = pd.merge(
    master_df,
    medications_count,
    on='visit_id',
    how='left'
)
master_df['medication_count'].fillna(0, inplace=True)
print(f"after merging medication count: {len(master_df)} rows")

# merge all medications
master_df = pd.merge(
    master_df,
    all_medications,
    on='visit_id',
    how='left'
)
print(f"after merging all medications: {len(master_df)} rows")

print(f"\nfinal master dataset:")
print(f"  rows: {len(master_df)}")
print(f"  columns: {len(master_df.columns)}")


                            creating master dataset                             

starting with patient-visits: 2481 rows
after merging diagnosis count: 2481 rows
after merging primary diagnosis: 2481 rows
after merging all diagnoses: 2481 rows
after merging medication count: 2481 rows
after merging all medications: 2481 rows

final master dataset:
  rows: 2481
  columns: 19


## 6. merge staff information

In [7]:
# merge staff information if available
print_section_header("merging staff information")

if 'attending_physician_id' in master_df.columns or 'staff_id' in visits_df.columns:
    # determine the join column
    join_col = 'attending_physician_id' if 'attending_physician_id' in master_df.columns else 'staff_id'
    
    if join_col in master_df.columns and 'staff_id' in staff_df.columns:
        master_df = pd.merge(
            master_df,
            staff_df,
            left_on=join_col,
            right_on='staff_id',
            how='left',
            suffixes=('', '_staff')
        )
        print(f"staff information merged: {len(master_df)} rows")
    else:
        print("staff information columns not found for merging")
else:
    print("no staff id column available in visits data")


                           merging staff information                            

staff information merged: 2481 rows


## 7. handle missing values in merged dataset

In [8]:
# handle missing values in merged dataset
print_section_header("handling missing values in master dataset")

# display missing values
missing_summary = master_df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("columns with missing values:")
for col, count in missing_summary.items():
    pct = (count / len(master_df)) * 100
    print(f"  {col}: {count} ({pct:.2f}%)")

# fill specific missing values
if 'diagnosis_count' in master_df.columns:
    master_df['diagnosis_count'].fillna(0, inplace=True)

if 'medication_count' in master_df.columns:
    master_df['medication_count'].fillna(0, inplace=True)

print(f"\nmissing values after handling: {master_df.isnull().sum().sum()}")


                   handling missing values in master dataset                    

columns with missing values:
  all_medications: 750 (30.23%)
  primary_diagnosis: 480 (19.35%)
  all_diagnoses: 480 (19.35%)

missing values after handling: 1710


## 8. save integrated datasets

In [9]:
# save master dataset and separate entity tables
print_section_header("saving integrated datasets")

# save master dataset
save_dataframe(master_df, PREPROCESSED_DATA_DIR / 'master_patient_visits.csv')
print(f"✓ master dataset saved: {len(master_df)} rows, {len(master_df.columns)} columns")

# save curated entity tables
save_dataframe(diagnoses_df, PREPROCESSED_DATA_DIR / 'curated_diagnoses.csv')
print(f"✓ curated diagnoses saved: {len(diagnoses_df)} rows")

save_dataframe(medications_df, PREPROCESSED_DATA_DIR / 'curated_medications.csv')
print(f"✓ curated medications saved: {len(medications_df)} rows")

save_dataframe(staff_df, PREPROCESSED_DATA_DIR / 'curated_staff.csv')
print(f"✓ curated staff saved: {len(staff_df)} rows")

2025-11-10 17:48:28,915 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\master_patient_visits.csv
2025-11-10 17:48:28,930 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\curated_diagnoses.csv
2025-11-10 17:48:28,947 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\curated_medications.csv
2025-11-10 17:48:28,930 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\curated_diagnoses.csv
2025-11-10 17:48:28,947 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\curated_medications.csv



                           saving integrated datasets                           

✓ master dataset saved: 2481 rows, 24 columns
✓ curated diagnoses saved: 7998 rows


2025-11-10 17:48:28,947 - root - INFO - saved dataframe to d:\Github Desktop\Python\Hospital Data Curation\data\preprocessed\curated_staff.csv


✓ curated medications saved: 5324 rows
✓ curated staff saved: 500 rows


## 9. generate integration report

In [10]:
# create integration report
print_section_header("integration summary report")

integration_report = {
    'source_datasets': {
        'patients': len(patients_df),
        'visits': len(visits_df),
        'diagnoses': len(diagnoses_df),
        'medications': len(medications_df),
        'staff': len(staff_df)
    },
    'master_dataset': {
        'total_rows': len(master_df),
        'total_columns': len(master_df.columns),
        'unique_patients': master_df['patient_id'].nunique() if 'patient_id' in master_df.columns else 0,
        'unique_visits': master_df['visit_id'].nunique() if 'visit_id' in master_df.columns else 0,
        'missing_values': master_df.isnull().sum().sum()
    }
}

# display report
print("\nsource datasets:")
for dataset, count in integration_report['source_datasets'].items():
    print(f"  {dataset}: {count} rows")

print("\nmaster dataset:")
for metric, value in integration_report['master_dataset'].items():
    print(f"  {metric}: {value}")

# display column list
print("\nmaster dataset columns:")
for i, col in enumerate(master_df.columns, 1):
    print(f"  {i}. {col}")


                           integration summary report                           


source datasets:
  patients: 3000 rows
  visits: 2481 rows
  diagnoses: 7998 rows
  medications: 5324 rows
  staff: 500 rows

master dataset:
  total_rows: 2481
  total_columns: 24
  unique_patients: 1692
  unique_visits: 2481
  missing_values: 1710

master dataset columns:
  1. patient_id
  2. name
  3. dob
  4. gender
  5. contact_number
  6. email
  7. address
  8. visit_id
  9. admission_date
  10. discharge_date
  11. admission_type
  12. hospital_unit_id
  13. attending_physician_id
  14. length_of_stay
  15. diagnosis_count
  16. primary_diagnosis
  17. all_diagnoses
  18. medication_count
  19. all_medications
  20. staff_id
  21. name_staff
  22. designation
  23. unit_id
  24. joining_date


## summary

data integration completed:
- ✓ patients and visits merged successfully
- ✓ diagnoses aggregated per visit
- ✓ medications aggregated per visit
- ✓ staff information integrated
- ✓ master dataset created with all entities
- ✓ all integrated datasets saved to `data/preprocessed/`

next phase: data transformation and feature engineering